# Laboratoire 4 : Développement d’un système intelligent
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexandre Laroche - LARA12078907<br>Marc-Antoine Charland - CHAM16059609<br>Jonathan Croteau-Dicaire - CROJ10109402    |
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Été 2019                                            |
| Groupe                | 02                                                      |
| Numéro du laboratoire | TP-04                                                   |
| Professeur            | Prof. Alessandro L. Koarich                             |
| Chargé de laboratoire | Pierre-Luc Delisle                                                     |
| Date                  | 5 août 2019 (23h55)                                                    |

In [14]:
from os import path

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.regularizers import l1

from src import constants

In [2]:
def load_feature_set():

    file_path = path.join(constants.RAW_TAGGED_FEATURE_SET_PATH, "msd-marsyas_dev_new/msd-marsyas_dev_new.csv")
    
    ftrs = np.array(pd.read_csv(file_path, header=None).values[:,2:-1])
    lbls = np.array(pd.read_csv(file_path, header=None).values[:,-1])
    
    return ftrs, lbls

def scale_features(ftrs):
    
    standardscaler = StandardScaler()
    ftrs_scld = standardscaler.fit_transform(ftrs)
    
    return ftrs_scld

In [3]:
# Load the features and the labels
raw_ftrs, raw_lbls = load_feature_set()

ftrs = scale_features(raw_ftrs)

print(ftrs.shape)

(179555, 124)


In [4]:
print([round(a, 3) for a in max(ftrs, key=lambda x: x[1])])

[17.438, 20.63, 7.722, -1.581, -1.314, -1.21, 3.325, -0.239, -1.969, 0.958, -5.28, 0.136, 0.776, -6.516, 5.989, -6.262, 2.074, -1.255, -0.844, -0.474, -1.235, -1.254, -1.253, -0.731, 1.572, -0.944, -1.235, -1.239, -1.242, 15.63, 0.115, 0.55, 0.391, -1.568, -1.045, -1.164, -1.413, -1.556, -1.708, -1.511, -1.646, -1.838, -1.826, -1.849, -1.608, -1.284, -1.129, -1.758, -1.221, -1.07, -0.92, -1.224, -1.212, -1.211, -1.112, -0.694, -1.155, -1.189, -1.194, -1.203, 1.586, 0.01, 4.135, 1.115, -2.151, 1.072, -0.495, -0.51, -0.378, -1.094, -0.449, -0.279, -0.67, -0.934, -1.151, -0.926, -1.056, -0.994, -1.413, -1.184, -0.866, -0.55, -1.207, -1.201, -1.21, -0.805, 1.029, -0.977, -1.176, -1.175, -1.165, 3.405, 0.013, 2.122, -0.381, -2.077, 0.849, 1.398, -1.256, -1.11, -1.971, -1.002, -1.444, -1.171, -1.42, -1.658, -0.801, -0.404, -0.788, -1.205, -1.09, -0.989, -0.889, -1.127, -1.121, -1.131, -1.034, -0.614, -1.088, -1.105, -1.095, -1.082, 2.223, -0.001]


In [38]:
def create_model():

    regularization = 0.00000000001
    
    inputs = Input(shape=(124,), name="inputs")

    encoded = Dense(
        120, activation='relu', activity_regularizer=l1(regularization)
    )(inputs)
    
    #encoded = BatchNormalization()(encoded)
    
    encoded = Dense(
        115, activation='relu', activity_regularizer=l1(regularization)
    )(encoded)
    
    encoded = Dropout(0.05)(encoded)
    
    encoded = Dense(
        112, activation='relu', activity_regularizer=l1(regularization)
    )(encoded)
    
    bottleneck = Dense(110, activation='relu', name="bottleneck")(encoded)
    
    decoded = Dense(
        112, activation='relu', activity_regularizer=l1(regularization)
    )(bottleneck)
    
    decoded = Dropout(0.05)(decoded)
    
    decoded = Dense(
        115, activation='relu', activity_regularizer=l1(regularization)
    )(bottleneck)
    
    #decoded = BatchNormalization()(decoded)
    
    decoded = Dense(
        120, activation='relu', activity_regularizer=l1(regularization)
    )(decoded)

    outputs = Dense(124, activation='linear')(decoded)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(lr=0.001),
        loss='mean_squared_error'
    )
    
    return model, inputs, bottleneck

In [8]:
experiment_number = 108

In [45]:
experiment_number += 1

run_name = 'autoencoderdecoder_marsyas_' + str(experiment_number)
model_save_path = constants.MODELS_PATH + run_name + '.h5'

tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
model_ckeckpt = ModelCheckpoint(filepath=model_save_path, monitor='val_loss', save_best_only=True, verbose=0)

model, inputs_layer, bottleneck = create_model()

model.fit(
    x=ftrs, 
    y=ftrs,
    epochs=50,
    batch_size=500,
    validation_split=0.2,
    callbacks=[
        tk_board,
        early_stop,
        model_ckeckpt
    ],
    shuffle=True,
    verbose=2
)

Train on 143644 samples, validate on 35911 samples
Epoch 1/50
 - 9s - loss: 0.3337 - val_loss: 0.1869
Epoch 2/50
 - 2s - loss: 0.2144 - val_loss: 0.1189
Epoch 3/50
 - 2s - loss: 0.2127 - val_loss: 0.1069
Epoch 4/50
 - 2s - loss: 0.2221 - val_loss: 0.1855
Epoch 5/50
 - 2s - loss: 0.1877 - val_loss: 0.0950
Epoch 6/50
 - 2s - loss: 0.1935 - val_loss: 0.1081
Epoch 7/50
 - 2s - loss: 0.1636 - val_loss: 0.0907
Epoch 8/50
 - 2s - loss: 0.1614 - val_loss: 0.2314
Epoch 9/50
 - 2s - loss: 0.1368 - val_loss: 0.1013
Epoch 10/50
 - 2s - loss: 0.3863 - val_loss: 0.0854
Epoch 11/50
 - 2s - loss: 0.1463 - val_loss: 0.0769
Epoch 12/50
 - 2s - loss: 0.1350 - val_loss: 0.0752
Epoch 13/50
 - 2s - loss: 0.1377 - val_loss: 0.0719
Epoch 14/50
 - 2s - loss: 0.1265 - val_loss: 0.0707
Epoch 15/50
 - 2s - loss: 0.1780 - val_loss: 0.2879
Epoch 16/50
 - 2s - loss: 0.2064 - val_loss: 0.0892
Epoch 17/50
 - 2s - loss: 0.1479 - val_loss: 0.0747
Epoch 18/50
 - 2s - loss: 0.1210 - val_loss: 0.0688
Epoch 19/50
 - 2s - lo

In [46]:
save_path = constants.MODELS_PATH + 'autoencoder_marsyas_' + str(experiment_number) + '.h5'

autoencoder = Model(inputs=inputs_layer, outputs=bottleneck)
autoencoder.save(save_path)